In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
demo = pd.read_csv("concrete.csv")
X=demo.drop('strength',axis=1)
y=demo['strength']
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size= .20,random_state=10)
lr=LinearRegression()
lr.fit(Xtrain,ytrain)
print("Training R2")
print(lr.score(Xtrain,ytrain))
print("Testing R2")
print(lr.score(Xtest,ytest))

Training R2
0.6374354064424276
Testing R2
0.5022375995887234


In [3]:
scoresdt = cross_val_score(lr, Xtrain, ytrain, cv=10)

In [4]:
scoresdt

array([0.52571613, 0.71904056, 0.62260097, 0.64177363, 0.71449256,
       0.70070694, 0.58346583, 0.52009653, 0.65254622, 0.50414244])

In [5]:
demo = pd.read_csv("concrete.csv")
X=demo.drop('strength',axis=1)
y=demo['strength']
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size= .20,random_state=10)
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
poly = PolynomialFeatures(degree=3)
pca = PCA(n_components=130)
sc=StandardScaler()
scaledXtrain = sc.fit_transform(Xtrain)
scaledXtest = sc.transform(Xtest)
polyscaledXtrain = poly.fit_transform(scaledXtrain)
polyscaledXtest = poly.transform(scaledXtest)
pcaXtrain = pca.fit_transform(polyscaledXtrain)
pcaXtest = pca.transform(polyscaledXtest)
lr=LinearRegression()
scoresdt = cross_val_score(lr, polyscaledXtrain, ytrain, cv=10)
lrpca=LinearRegression()
scoresdtpca = cross_val_score(lrpca, pcaXtrain, ytrain, cv=10)

In [6]:
print("Before PCA")
print(np.mean(scoresdt))
print("After PCA")
print(np.mean(scoresdtpca))

Before PCA
0.8332261105080552
After PCA
0.849279741989208


In [7]:
#Bias Variance Error Calculation
print("Bias Errors (1-Accuracy/R2)")
print(1-np.mean(scoresdt))
print("Variance Errors")
print(np.var(scoresdt))

Bias Errors (1-Accuracy/R2)
0.16677388949194483
Variance Errors
0.0029530298639429083


In [22]:
#With Pipeline

import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
demo = pd.read_csv("concrete.csv")
X=demo.drop('strength',axis=1)
y=demo['strength']
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size= .20,random_state=10)
pipe = Pipeline((
("scaler", StandardScaler()),
("poly",PolynomialFeatures(degree=3)),
("pca",PCA(n_components=130)),
("rf", LinearRegression()),
))
scoresdtpipe = cross_val_score(pipe, Xtrain, ytrain, cv=10)
print("After PCA Avg Cross Val R2")
print(np.mean(scoresdtpipe))

After PCA Avg Cross Val R2
0.8506251915803915


In [23]:
print("Final model R2 on test data after fitting only on train data")
pipe.fit(Xtrain,ytrain)
print(pipe.score(Xtest,ytest))

Final model R2 on test data after fitting only on train data
0.8914371124423256


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
demo = pd.read_csv("concrete.csv")
X=demo.drop('strength',axis=1)
y=demo['strength']
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size= .20,random_state=10)
pipe = Pipeline((
("scaler", StandardScaler()),
("poly",PolynomialFeatures()),
("pca",PCA()),
("rf", LinearRegression()),
))

param_grid = {
    'poly__degree' : [2,3],
    'pca__n_components': [15, 30]
}
search = GridSearchCV(pipe, param_grid,cv=5,verbose=2)
#print(search.best_params_)
#scoresdtpipe = cross_val_score(search, Xtrain, ytrain, cv=10)
#print("After PCA Avg Cross Val R2")
#print(np.mean(scoresdtpipe))

In [3]:
search.fit(Xtrain,ytrain)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] pca__n_components=15, poly__degree=2 ............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s

[CV] ............. pca__n_components=15, poly__degree=2, total=   0.6s
[CV] pca__n_components=15, poly__degree=2 ............................


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


[CV] ............. pca__n_components=15, poly__degree=2, total=   0.5s
[CV] pca__n_components=15, poly__degree=2 ............................


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


[CV] ............. pca__n_components=30, poly__degree=3, total=   0.8s
[CV] pca__n_components=30, poly__degree=3 ............................


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   18.5s finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


[CV] ............. pca__n_components=30, poly__degree=3, total=   1.2s


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=(('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('poly', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('rf', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)))),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'poly__degree': [2, 3], 'pca__n_components': [15, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [4]:
search.best_params_

{'pca__n_components': 30, 'poly__degree': 2}

In [5]:
search.best_score_

0.7441488434687544

In [6]:
search.cv_results_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\util

{'mean_fit_time': array([0.44348869, 1.42260141, 0.39216037, 1.21022639]),
 'std_fit_time': array([0.16728065, 0.47051136, 0.05527527, 0.24442084]),
 'mean_score_time': array([0.02518468, 0.01889124, 0.02650347, 0.03096452]),
 'std_score_time': array([0.01097886, 0.00621531, 0.0093293 , 0.01256502]),
 'param_pca__n_components': masked_array(data=[15, 15, 30, 30],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_poly__degree': masked_array(data=[2, 3, 2, 3],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'pca__n_components': 15, 'poly__degree': 2},
  {'pca__n_components': 15, 'poly__degree': 3},
  {'pca__n_components': 30, 'poly__degree': 2},
  {'pca__n_components': 30, 'poly__degree': 3}],
 'split0_test_score': array([0.36811134, 0.28509247, 0.71358968, 0.50240928]),
 'split1_test_score': array([0.24392624, 0.34395084, 0.76031715, 0.5144771 ]),
 'split2_test_sc

In [8]:
results = search.cv_results_
for param, score_mean, score_sd in zip(results['params'], results['mean_test_score'], results['std_test_score']):
    print(param, round(score_mean, 4), round(score_sd, 4))

{'pca__n_components': 15, 'poly__degree': 2} 0.3278 0.0707
{'pca__n_components': 15, 'poly__degree': 3} 0.3506 0.0369
{'pca__n_components': 30, 'poly__degree': 2} 0.7441 0.0195
{'pca__n_components': 30, 'poly__degree': 3} 0.5222 0.029


In [9]:
results = search.cv_results_
for param, score_mean, score_sd in zip(results['params'], results['mean_train_score'], results['std_train_score']):
    print(param, round(score_mean, 4), round(score_sd, 4))

{'pca__n_components': 15, 'poly__degree': 2} 0.3581 0.0227
{'pca__n_components': 15, 'poly__degree': 3} 0.4004 0.0184
{'pca__n_components': 30, 'poly__degree': 2} 0.7783 0.0027
{'pca__n_components': 30, 'poly__degree': 3} 0.5832 0.0183
